In [9]:
import pandas as pd
import base64
import pymongo

generar = False


In [10]:
if(generar):
   #Obtención de datos desde BD
   SERVER_IP = "192.168.22.69"
   client = pymongo.MongoClient("mongodb://{}:27017".format(SERVER_IP))

   database = client["retoYAE"]

   collection = database["images"]

   docs = collection.find()

   myData = []
   for i in docs:
      myData.append(i)

   for i in range(0,5):
      print(myData[i])

   df = pd.DataFrame(myData)

   #Generación de imágenes en local
   lista = []
   for i in df['content']:
      lista.append(bytes.fromhex(i))

   df["content"] = lista

   if(generar):
      for index, row in df.iterrows():
         with open("./img/"+ row["_id"] +".png","wb") as f:
            f.write(row["content"])


In [11]:
#Generar los randoms
import random
import os

directorios = os.listdir("./img/")
filesAmount = len(directorios)-1
randNums = []
for i in range(1,int(filesAmount*0.2)):
    num =random.randint(1, filesAmount)
    while(num in randNums):
        num =random.randint(1, filesAmount)    
    randNums.append(num)
randNums = sorted(randNums)
#Pasar la lista de los randoms a la carpeta test
for i in randNums:
    os.rename("./img/"+directorios[i],"./predict/"+directorios[i])


In [12]:
img_route = "./img/"

import numpy as np # linear algebra
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
%matplotlib inline 
import matplotlib.pyplot as plt
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import cv2
import string
import numpy as np
from PIL import Image
from keras_preprocessing.image import img_to_array
import seaborn

#Init main values
symbols ="aAbBcCdDeEfFgGhHjJkKlLmMnNpPqQrRsStTuUvVwWxXyY23456789"  # Simbolos del captcha
num_symbols = len(symbols)
img_shape = (50, 200, 1)

def convert_image(route):
    img = cv2.imread(route)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    # Posiciones de pixeles negros
    black_pixels = cv2.findNonZero(threshold)
    # Crear mascara para quedarnos los pixeles negros
    mask = gray < threshold
    # Colorear pixeles en esos lugares por un color elegido
    img[mask] = (255, 255, 255)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def create_model():

    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 5 branches from it. Each branch will predict one letter
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(128, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    
    # Compile model and return it
    model = Model(img, outs)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

def preprocess_data():
    n_samples = len(os.listdir(img_route))
    X = np.zeros((n_samples, 50, 200, 1)) #1070*50*200
    y = np.zeros((5, n_samples, num_symbols)) #5*1070*36

    for i, pic in enumerate(os.listdir(img_route)):
        # Read image as grayscale
        if pic == '.config':
          continue
        img = convert_image(os.path.join(img_route, pic))
        pic_target = pic[:-4]
        if len(pic_target) < 6:
            # Scale and reshape image
            img = (img/255)
            img = np.reshape(img, (50, 200, 1))
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs

    return X, y

X, y = preprocess_data()

testingUpperMargin = int(filesAmount*0.8)
X_train, y_train = X[:testingUpperMargin], y[:, :testingUpperMargin]
X_test, y_test = X[testingUpperMargin:], y[:, testingUpperMargin:]

model=create_model();
model.summary();

hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=128, epochs=50, verbose = 0)





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 200, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 50, 200, 16)  160         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 25, 100, 16)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 25, 100, 32)  4640        ['max_pooling2d[0][0]']      

In [13]:
def predict(filepath):
    img = convert_image(filepath)
    if img is not None:
        img = (img/255.0)
    else:
        print("Error with the image");
    #Crear el "resultado" de la imagen con la prediccion del modelo
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 54)) 
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt

score= model.evaluate(X_test,[y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]],verbose=1)

with open("./output/Stats.txt", "w") as f:
     dic = dict(zip(model.metrics_names, score))
     for i in dic.keys():
          f.write("{};{}\n".format(i, dic[i]))  

1/1 [==============================] - 1s 884ms/step - loss: 9.8148e-06 - dense_1_loss: 7.9473e-08 - dense_3_loss: 0.0000e+00 - dense_5_loss: 9.7353e-06 - dense_7_loss: 0.0000e+00 - dense_9_loss: 0.0000e+00 - dense_1_accuracy: 1.0000 - dense_3_accuracy: 1.0000 - dense_5_accuracy: 1.0000 - dense_7_accuracy: 1.0000 - dense_9_accuracy: 1.0000


In [14]:
with open("./output/resultados.csv", "w") as f:
    f.write("real;predicted;valid\n")
    for i in os.listdir("./predict/"):
            predicted =  predict("./predict/"+i)
            real = i.split(".")[0] 
            if real == predicted:    
                f.write(f"{real};{predicted};1\n")       
            else:    
                 f.write(f"{real};{predicted};0\n")   
          

1/1 [==============================] - 0s 29ms/step


In [15]:
directorios = os.listdir("./predict/")
filesAmount = len(directorios)
for i in directorios:
    os.rename("./predict/"+i,"./img/"+i)



In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()